In this notebook we will explore thed data to find ways to generate more revenue for taxi cab drivers. 
Our model wil Identify whether a customers is a generous tipper 


## Feature Engineering

In [10]:
#  Imports 
import pandas as pd
import numpy as np  

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Model Selection 
from sklearn.model_selection import GridSearchCV, train_test_split

# Machine Learning Models
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier , plot_importance

# Evaluation Metrics (classification)
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay

# Utilities
import warnings
warnings.filterwarnings("ignore")


In [11]:
pd.set_option('display.max_columns', None)

In [23]:
df0 = pd.read_csv('2017_Yellow_Taxi_Trip_Data.csv', index_col=0)
# NYC taxi fare predictions from the multiple linear regression model
nyc_preds_means = pd.read_csv('nyc_taxi_fare_predictions_mlr.csv', index_col=0)

In [28]:
nyc_preds_means.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22699 entries, 22.847222222222214 to 9.405555555555557
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   mean_distance   22699 non-null  float64
 1   predicted_fare  22699 non-null  float64
dtypes: float64(2)
memory usage: 532.0 KB


In [26]:
df0.head()
# print(df0.shape)
# print(nyc_preds_means.shape)
#print(df0.index)
#print(nyc_preds_means.index)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
24870114,2,03/25/2017 8:55:43 AM,03/25/2017 9:09:47 AM,6,3.34,1,N,100,231,1,13.0,0.0,0.5,2.76,0.0,0.3,16.56
35634249,1,04/11/2017 2:53:28 PM,04/11/2017 3:19:58 PM,1,1.80,1,N,186,43,1,16.0,0.0,0.5,4.00,0.0,0.3,20.80
106203690,1,12/15/2017 7:26:56 AM,12/15/2017 7:34:08 AM,1,1.00,1,N,262,236,1,6.5,0.0,0.5,1.45,0.0,0.3,8.75
38942136,2,05/07/2017 1:17:59 PM,05/07/2017 1:48:14 PM,1,3.70,1,N,188,97,1,20.5,0.0,0.5,6.39,0.0,0.3,27.69
30841670,2,04/15/2017 11:32:20 PM,04/15/2017 11:49:03 PM,1,4.37,1,N,4,112,2,16.5,0.5,0.5,0.00,0.0,0.3,17.80


In [27]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22699 entries, 24870114 to 17208911
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorID               22699 non-null  int64  
 1   tpep_pickup_datetime   22699 non-null  object 
 2   tpep_dropoff_datetime  22699 non-null  object 
 3   passenger_count        22699 non-null  int64  
 4   trip_distance          22699 non-null  float64
 5   RatecodeID             22699 non-null  int64  
 6   store_and_fwd_flag     22699 non-null  object 
 7   PULocationID           22699 non-null  int64  
 8   DOLocationID           22699 non-null  int64  
 9   payment_type           22699 non-null  int64  
 10  fare_amount            22699 non-null  float64
 11  extra                  22699 non-null  float64
 12  mta_tax                22699 non-null  float64
 13  tip_amount             22699 non-null  float64
 14  tolls_amount           22699 non-null  float64
 1

In [6]:
nyc_preds_means.head()

,mean_distance,predicted_fare
mean_duration,,
22.847222,3.521667,16.434245
24.470370,3.108889,16.052218
7.250000,0.881429,7.053706
30.250000,3.700000,18.731650
14.616667,4.435000,15.845642


In [19]:
#merge the two dataframes on the index
df0 = df0.merge(nyc_preds_means, left_index=True, right_index=True)


In [20]:
df0.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,mean_distance,predicted_fare
